# Notebook 03: Building the RAG Agent
## Introduction

In this notebook, we will focus on constructing a fully functional Retrieval-Augmented Generation (RAG) Agent. The RAG agent combines various components—like the language model (LLM), vector store, and conversation context—into a unified pipeline to retrieve relevant information from external documents and generate contextually enriched responses.

We will walk through the final steps of assembling the RAG agent and demonstrate how to execute queries using the agent.

## Review of Components

Before diving into the full implementation of the RAG agent, let’s briefly recap the key components:

- **Language Model (LLM)**: Generates responses based on the input query.
- **Vector Store**: Holds external documents and allows retrieval of relevant documents based on the query.
- **Conversation Context**: Maintains the dialogue context, ensuring that responses are coherent and relevant to previous interactions.

In this notebook, we will integrate these components into the RagAgent.

## Setting Up the Vector Store and Adding Documents

The first step is to set up the vector store, which will hold our external knowledge in the form of documents. The RAG agent will use this store to retrieve relevant information during query processing.

In [35]:
from swarmauri.documents.concrete.Document import Document
from swarmauri.vector_stores.concrete.TfidfVectorStore import TfidfVectorStore

# Initialize the vector store
vector_store = TfidfVectorStore()

# Sample documents containing knowledge
documents = [
    Document(content="Their sister's name is Jane."),
    Document(content="Their mother's name is Jean."),
    Document(content="Their father's name is Joseph."),
    Document(content="Their grandfather's name is Alex."),
]

# Add documents to the vector store
vector_store.add_documents(documents)

# Verify the documents have been added
print(f"{len(vector_store.documents)} documents added to the vector store.")


4 documents added to the vector store.


## Configuring the Conversation Context

Next, we set up the conversation context to ensure the agent can handle multi-turn dialogues. This context helps the agent remember previous user queries and system responses.

In [36]:
from swarmauri.conversations.concrete.MaxSystemContextConversation import MaxSystemContextConversation
from swarmauri.messages.concrete.SystemMessage import SystemMessage
from swarmauri.messages.concrete.HumanMessage import HumanMessage 

# Create a system message
system_context = SystemMessage(content="Your name is Jeff.")

# Initialize the conversation
conversation = MaxSystemContextConversation(system_context=system_context, max_size=4)

# Add a user message
user_message = HumanMessage(content="What is my name?")
conversation.add_message(user_message)

# Print the current conversation context
print("Current conversation history:")
for message in conversation.history:
    print(message.content)

Current conversation history:
Your name is Jeff.
What is my name?


## Integrating the Language Model (LLM)

The GroqModel (our chosen LLM) will generate responses based on both the retrieved documents and the conversation context. Let's initialize the LLM and integrate it into our RAG agent.

### Importing Libraries and Loading Environment Variables

In [31]:
import os
from swarmauri.llms.concrete.GroqModel import GroqModel as LLM
from swarmauri.conversations.concrete.Conversation import Conversation
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

### Setting the API Key

In [37]:
API_KEY = os.getenv("GROQ_API_KEY")

# Check if the API key is set
if not API_KEY:
    print("API key is not set. Please set the GROQ_API_KEY environment variable.")

### Function to Get Allowed Models

In [38]:
# Function to get allowed models, filtering out failing ones
def get_allowed_models(llm):
    failing_llms = [
        "llama3-70b-8192",
        "llama-3.2-90b-text-preview",
        "mixtral-8x7b-32768",
        "llava-v1.5-7b-4096-preview",
        "llama-guard-3-8b",
    ]
    return [model for model in llm.allowed_models if model not in failing_llms]

### Initializing the GroqModel and Displaying Information

In [39]:
# Initialize the GroqModel
llm = LLM(api_key=API_KEY)

# Print model information
print(f"Resource: {llm.resource}")
print(f"Type: {llm.type}")
print(f"Default Name: {llm.name}")

Resource: LLM
Type: GroqModel
Default Name: gemma-7b-it


### Retrieving Allowed Models

In [40]:
# Get allowed models
allowed_models = get_allowed_models(llm)
print("Allowed Models:", allowed_models)

Allowed Models: ['gemma-7b-it', 'gemma2-9b-it', 'llama-3.1-70b-versatile', 'llama-3.1-8b-instant', 'llama-3.2-11b-text-preview', 'llama-3.2-1b-preview', 'llama-3.2-3b-preview', 'llama3-8b-8192', 'llama3-groq-70b-8192-tool-use-preview', 'llama3-groq-8b-8192-tool-use-preview']


### Example Usage with No System Context

In [42]:
# Example usage with no system context
llm.name = allowed_models[0]

# Create a conversation
conversation = Conversation()

# Add a human message
input_data = "Hello"
human_message = HumanMessage(content=input_data)
conversation.add_message(human_message)

# Predict response
llm.predict(conversation=conversation)
prediction = conversation.get_last().content
print(f"Prediction with no system context for {llm.name}: {prediction}")

Prediction with no system context for gemma-7b-it: Hello! 👋 It's great to hear from you. What can I do for you today? 😊


### Example Usage with a System Context

In [48]:
# Example usage with a system context
system_context = 'Human History'
conversation = MaxSystemContextConversation(system_context=SystemMessage(content=system_context), max_size=2)
system_message = SystemMessage(content=system_context)
conversation.add_message(HumanMessage(content="human1"))

human_message = HumanMessage(content="Hi")
conversation.add_message(human_message)

# Predict response
llm.predict(conversation=conversation)
prediction = conversation.get_last().content
print(f"Prediction with system context for {llm.name}: {prediction}")

Prediction with system context for gemma-7b-it: **Human History (Human1)**

**Unit Overview:**

Human history explores the remarkable journey of our species from our origins in Africa to the diverse civilizations that have shaped the world. This unit examines the key events, developments, and cultural transformations that have defined human existence.

**Topics Covered:**

**Part 1: Origins and Evolution**

* Paleolithic Era: Stone tools, human evolution, and the first civilizations
* Neolithic Revolution: Agricultural innovations and the rise of settled life
* The Rise of Complex Societies: Early cities and states in Mesopotamia, Egypt, and the Indus Valley

**Part 2: Classical Eras**

* Ancient Greece: Political systems, philosophy, and the development of democracy
* Roman Republic and Empire: Political and military achievements, infrastructure, and literary contributions
* China: Imperial expansion, philosophical advancements, and technological innovations

**Part 3: Medieval and Re

### Complete Script for Initializing, Configuring, and Using GroqModel Examples

In [49]:
import os
from swarmauri.llms.concrete.GroqModel import GroqModel as LLM
from swarmauri.conversations.concrete.Conversation import Conversation
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

API_KEY = os.getenv("GROQ_API_KEY")
# Check if the API key is set
if not API_KEY:
    print("API key is not set. Please set the GROQ_API_KEY environment variable.")

# Function to get allowed models, filtering out failing ones
def get_allowed_models(llm):
    failing_llms = [
        "llama3-70b-8192",
        "llama-3.2-90b-text-preview",
        "mixtral-8x7b-32768",
        "llava-v1.5-7b-4096-preview",
        "llama-guard-3-8b",
    ]
    return [model for model in llm.allowed_models if model not in failing_llms]

# Initialize the GroqModel
llm = LLM(api_key=API_KEY)

# Print model information
print(f"Resource: {llm.resource}")
print(f"Type: {llm.type}")
print(f"Default Name: {llm.name}")

# Get allowed models
allowed_models = get_allowed_models(llm)
print("Allowed Models:", allowed_models)

# Example usage with no system context

llm.name = allowed_models[0]

# Create a conversation
conversation = Conversation()

# Add a human message
input_data = "Hello"
human_message = HumanMessage(content=input_data)
conversation.add_message(human_message)

# Predict response
llm.predict(conversation=conversation)
prediction = conversation.get_last().content
print(f"Prediction with no system context for {llm.name}: {prediction}")

# Example usage with a system context
system_context = 'Human History'
conversation = MaxSystemContextConversation(system_context=SystemMessage(content=system_context), max_size=2)
system_message = SystemMessage(content=system_context)
conversation.add_message(HumanMessage(content="human1"))

human_message = HumanMessage(content="Hi")
conversation.add_message(human_message)

# Predict response
llm.predict(conversation=conversation)
prediction = conversation.get_last().content
print(f"Prediction with system context for {llm.name}: {prediction}")

Resource: LLM
Type: GroqModel
Default Name: gemma-7b-it
Allowed Models: ['gemma-7b-it', 'gemma2-9b-it', 'llama-3.1-70b-versatile', 'llama-3.1-8b-instant', 'llama-3.2-11b-text-preview', 'llama-3.2-1b-preview', 'llama-3.2-3b-preview', 'llama3-8b-8192', 'llama3-groq-70b-8192-tool-use-preview', 'llama3-groq-8b-8192-tool-use-preview']
Prediction with no system context for gemma-7b-it: Hello! 👋 I'm happy to hear from you. How can I help you today? 😊
Prediction with system context for gemma-7b-it: **Human History (Human1)**

**Unit 1: Origins and Evolution**

* Origin and evolution of hominin species
* The Stone Age: Paleolithic and Mesolithic eras
* The development of agriculture and the transition to a sedentary lifestyle

**Unit 2: Ancient Civilizations**

* The rise and fall of ancient civilizations such as Mesopotamia, Egypt, and the Indus Valley Civilization
* The development of writing, technology, and art
* The impact of ancient civilizations on the modern world

**Unit 3: Classical E

## Building the RAG Agent
Finally, we will assemble all the components into the RagAgent. The agent will retrieve relevant documents from the vector store and use the language model to generate responses based on both the retrieved content and the conversation context.

In [50]:
from swarmauri.agents.concrete.RagAgent import RagAgent

# Initialize the RAG Agent by combining LLM, conversation, and vector store
rag_agent = RagAgent(
    llm=llm,
    conversation=conversation,
    system_context=system_context,
    vector_store=vector_store,
)

# Example query to the RAG agent
query = "What is the name of their grandfather?"
response = rag_agent.exec(query)

# Print the agent's response
print(f"RAG Agent Response: {response}")


RAG Agent Response: The name of their grandfather is Alex, as stated in the given sentence: "Their grandfather's name is Alex."


## Handling Queries with the RAG Agent
Now that the RAG agent is fully configured, we can test it with various queries. The RAG agent will retrieve documents from the vector store, interpret the conversation context, and generate informed responses.

In [51]:
# Test the agent with different queries
queries = [
    "What is the name of their mother?",
    "What is the name of their sister?",
    "Tell me more about their family."
]

for query in queries:
    response = rag_agent.exec(query)
    print(f"Query: {query}\nRAG Agent Response: {response}\n")


Query: What is the name of their mother?
RAG Agent Response: The name of their mother is Jean, as mentioned in the given information.

Query: What is the name of their sister?
RAG Agent Response: The name of their sister is Jane, as stated in the given sentence: "Their sister's name is Jane."

Query: Tell me more about their family.
RAG Agent Response: The provided text does not contain any further information about the individual's family beyond the names of their mother, father, grandfather, and sister. Therefore, I am unable to provide any additional details about their family structure, dynamics, or relationships.



## Notebook Metadata

In [52]:
import platform
import sys
from datetime import datetime

author_name = "Huzaifa Irshad " 
github_username = "irshadhuzaifa"

print(f"Author: {author_name}")
print(f"GitHub Username: {github_username}")

notebook_file = "Notebook_03_Building_RAG_Agent.ipynb"
try:
    last_modified_time = os.path.getmtime(notebook_file)
    last_modified_datetime = datetime.fromtimestamp(last_modified_time)
    print(f"Last Modified: {last_modified_datetime}")
except Exception as e:
    print(f"Could not retrieve last modified datetime: {e}")

print(f"Platform: {platform.system()} {platform.release()}")
print(f"Python Version: {sys.version}")

try:
    import swarmauri
    print(f"Swarmauri Version: {swarmauri.__version__}")
except ImportError:
    print("Swarmauri is not installed.")

Author: Huzaifa Irshad 
GitHub Username: irshadhuzaifa
Last Modified: 2024-10-28 13:59:29.671765
Platform: Windows 11
Python Version: 3.12.7 | packaged by Anaconda, Inc. | (main, Oct  4 2024, 13:17:27) [MSC v.1929 64 bit (AMD64)]
Swarmauri Version: 0.5.0
